In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import xgboost
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
import torch

import warnings
warnings.simplefilter('ignore')





In [ ]:
train = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/train.csv')
test = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/test.csv')


In [ ]:
train

In [ ]:
train['Province_State'].fillna("none",inplace=True)
train['ConfirmedCases'] = train['ConfirmedCases'].astype(int) 
train['Fatalities'] = train['Fatalities'].astype(int)

In [ ]:
test

In [ ]:
train['Date'] = pd.to_datetime(train['Date'])
train['Day']=train['Date'].dt.day
train['Month']=train['Date'].dt.month

In [ ]:
train.info(), test.info()

In [ ]:
lbl=LabelEncoder()
train['Province_State']=lbl.fit_transform(train['Province_State'])
train['Country_Region']=lbl.fit_transform(train['Country_Region'])

In [ ]:
train['Country_Region'].value_counts()

In [ ]:
train.describe()

In [ ]:
sns.heatmap(train.corr(), annot= True);

In [ ]:
X = train[['Province_State','Country_Region','Day','Month']]
X.head()

In [ ]:
y = train[['ConfirmedCases','Fatalities']]
y.head()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15)
#X_train = X.iloc[: 21597, :]
#X_val = X.iloc[21597:, :]
#y_train = y.iloc[: 21597, :]
#y_val = y.iloc[21597:, :]

In [ ]:
train.info()

In [ ]:
tree=DecisionTreeRegressor(criterion= 'mae')
#gs = GridSearchCV(tree, param_grid= {'max_depth' : np.arange (1, 8, 1), 'max_features' : np.arange(1, 5, 1)}, cv= 5, scoring= 'r2' )
tree.fit(X_train, y_train)
tree.score(X_val, y_val)


In [ ]:
tree=RandomForestRegressor(criterion= 'mae', n_estimators= 100)
#gs = GridSearchCV(tree, param_grid= {'max_depth' : np.arange (1, 8, 1), 'max_features' : np.arange(1, 5, 1)}, cv= 5, scoring= 'r2' )
tree.fit(X_train, y_train)
tree.score(X_val, y_val)

In [ ]:
tree = xgboost.XGBRegressor(n_estimators= 10000)
tree.fit(X_train, y_train.iloc[:, 0])
tree.score(X_val, y_val.iloc[:, 0])

In [ ]:
tree = xgboost.XGBRegressor(n_estimators= 1000)
tree.fit(X_train, y_train.iloc[:, 1])
tree.score(X_val, y_val.iloc[:, 1])

In [ ]:
pf = PolynomialFeatures(degree= 2)

In [ ]:
X_train_pf = pf.fit_transform(X_train, y_train)
X_val_pf = pf.transform(X_val)

In [ ]:
ridge = Ridge(alpha= 4.5,)
#GS = GridSearchCV(ridge, param_grid= {'alpha' : np.linspace (0.01, 10, 50)}, scoring= 'r2')
ridge.fit(X_train_pf, y_train)
ridge.score(X_val_pf, y_val)


In [ ]:
svr = SVR()
GS = GridSearchCV(svr, param_grid= {'kernel' : ['rbf', 'linear', 'poly', 'sigmoid']}, scoring= 'r2')
GS.fit(X_train, y_train.iloc[: , 0])
GS.score(X_val, y_val.iloc[:, 0])

In [ ]:
y_pred = tree.predict(X_val)

In [ ]:
plt.figure(figsize= (15, 10))
plt.scatter(np.arange(len (y_val)), abs(y_val.iloc[:, 0] - y_pred[:, 0])/ len(y_val))
#plt.yscale(np.arange(0, 100, 5))
plt.grid()
plt.title('Процент отклонения предсказанных данных от фактических')
plt.show()

In [ ]:
test['Date'] = pd.to_datetime(test['Date'])
test['Day']=test['Date'].dt.day
test['Month']=test['Date'].dt.month
test = test.drop(['ForecastId', 'Date'], axis= 1)

In [ ]:
 test

In [ ]:
test['Province_State'].fillna("none",inplace=True)

In [ ]:
lbl=LabelEncoder()
test['Province_State']=lbl.fit_transform(test['Province_State'])
test['Country_Region']=lbl.fit_transform(test['Country_Region'])# тест это Х тест

In [ ]:
X = train[['Province_State','Country_Region','Day','Month']]
X.head()

In [ ]:
y = train[['ConfirmedCases','Fatalities']]
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15)

In [ ]:
X_train = torch.FloatTensor(X_train.to_numpy())
X_test = torch.FloatTensor(X_test.to_numpy())
y_train = torch.FloatTensor(y_train.to_numpy())
y_test = torch.FloatTensor(y_test.to_numpy())

In [ ]:
X_train.dtype, y_train.dtype

In [ ]:
X_train.shape, X_test.shape, y_train, y_test

In [ ]:
X_train.shape

In [ ]:
class Covid19_Net(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(Covid19_Net, self).__init__()
        self.fc1 = torch.nn.Linear(4, n_hidden_neurons)
        self.ac1 = torch.nn.LeakyReLU()
        self.fc2 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.ac2 = torch.nn.LeakyReLU()
        self.fc3 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.ac3 = torch.nn.LeakyReLU()
        self.fc4 = torch.nn.Linear(n_hidden_neurons, 2)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.ac1(x)
        x = self.fc2(x)
        x = self.ac2(x)
        x = self.fc3(x)
        x = self.ac3(x)
        x = self.fc4(x)
        
        return x
    
covid19_net = Covid19_Net(50)

In [ ]:
loss = torch.nn.L1Loss()
optimizer = torch.optim.Adam(covid19_net.parameters(), lr= 0.01, weight_decay= 0.001)

In [ ]:
batch_size = 100

# test_accuracy_history = []
# test_loss_history = []

# X_test = X_test.to(device)
# y_test = y_test.to(device)

for epoch in range(10000):
    order = np.random.permutation(len(X_train))
    
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        
        X_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        
        preds = covid19_net.forward(X_batch)
        
        loss_value = loss(preds, y_batch)
        loss_value.backward()
        
        optimizer.step()
    if epoch % 25 == 0:
        test_preds = covid19_net.forward(X_test)
        print('epoch: {} || test_loss: {} \n'.format(epoch, loss(test_preds, y_test)))

In [ ]:
tree=DecisionTreeRegressor(criterion= 'mae')
tree.fit(X, y)
tree.score(X, y)



In [ ]:
tree_predict = tree.predict(test)

In [ ]:
tree_predict

In [ ]:
test

In [ ]:
test = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/test.csv')

In [ ]:
submit = pd.DataFrame(test["ForecastId"])

In [ ]:
submit['ConfirmedCases'] = tree_predict[:, 0]
submit['Fatalities'] = tree_predict[:, 1]

In [ ]:
submit.head(100)

In [ ]:
submit.to_csv('samplesubmission.csv', index = False)